<a href="https://colab.research.google.com/github/Arseniy-Polyakov/machine_learning_course/blob/main/Task_3_CNN_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данной работе будет проводиться классификация видеозаписей, где показывается русский жестовый язык (РЖЯ) на основе датасета [Slovo](https://www.kaggle.com/datasets/kapitanov/slovo) с помощью технологий CNN + RNN(LSTM)

Устанавливаем необходимые библиотеки для скачивания датасета через платформу kaggle, обработки данных и архитектуры модели сверточной и реккурентной нейронной сети

In [ ]:
!pip install json_repair

In [ ]:
import re
import os
import cv2
import kagglehub
import json_repair
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from moviepy.editor import VideoFileClip
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical

  if event.key is 'enter':



Проходим авторизацию на платформе для получения полного доступа к датасету

In [ ]:
kagglehub.login()

Устанавлием датасет в облако google colab

In [ ]:
slovo_dataset = kagglehub.dataset_download("kapitanov/slovo")
slovo_dataset

'/kaggle/input/slovo'

Пишем функцию для обработки видео по кадрам и их нормализации: приведения к единому формату, выберем как вариант (112, 112). Также пропишем функцию паддинга для фреймов

In [ ]:
def resize_with_padding(frame: list, target_size: tuple, padding_color: tuple) -> list:
  """
  The function for resizing and padding video frames (due to the different length and width of each frame)
  """
  try:
    h, w = frame.shape[:2]
    target_h, target_w = target_size

    scale = min(target_w / w, target_h / h)
    new_w, new_h = int(w * scale), int(h * scale)

    resized_frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)

    delta_w = target_w - new_w
    delta_h = target_h - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    padded_frame = cv2.copyMakeBorder(resized_frame, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
  except:
    print("ERROR")
  return padded_frame

In [ ]:
def extract_features(dir_path: str) -> list:
  """
  The function for parsing videos into frames and resizing every frame
  """
  dataset = []
  max_frames = 100
  videos = os.listdir("/content/samples/")
  for i in tqdm(range(len(videos))):
    frames = []
    clip = VideoFileClip(dir_path + videos[i])
    for frame in clip.iter_frames():
      if frame is None:
        print("None значение")
        break
      frame_resized = resize_with_padding(frame, (112, 112), (0, 0, 0))
      if frame_resized is None:
        print(f"There was no resizing {videos[i]}")
        break
      else:
        frame_resized = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        frame_resized = frame_resized / 255.0
        frames.append(frame_resized)
        if len(frames) >= max_frames:
          break
      while len(frames) < max_frames:
        frames.append(np.zeros((112, 112, 3)))
    dataset.append(np.array(frames))
  return dataset

In [ ]:
video_features = extract_features("/content/samples/")
for item in video_features:
  if item is None or np.isnan(item).any() or np.isnan(item).any() or np.isinf(item).any():
    print("None")


100%|██████████| 99/99 [00:18<00:00,  5.49it/s]


Парсим файл annotations из датасета, который содержит общую информацию, в том числе и перевод РЖЯ на русский вербальный

In [ ]:
with open("annotations.txt", "rt", encoding="utf-8") as file:
  text = file.read()

In [ ]:
text_splitted = re.split("\t|\n", text)
text = [text_splitted[i] for i in range(8, len(text_splitted), 7)]

In [ ]:
classes = len(set(text[:100]))
classes

33

Извлекаем категории для классификации, создаем на основе них массив

In [ ]:
labels_dict = {}
count = 0
labels = text[:100]
for i in tqdm(range(len(labels))):
  if labels[i] not in labels_dict:
    labels_dict[labels[i]] = count
    count += 1
y_labels = [[labels_dict[labels[i]]] for i in range(len(labels))]
y_labels_categorical = to_categorical(y_labels, num_classes=classes)
y_labels_categorical = np.array(y_labels_categorical)
y_labels_categorical

100%|██████████| 100/100 [00:00<00:00, 492867.69it/s]


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Прописываем слои модели CNN + RNN

In [ ]:
cnn_model = Sequential()
cnn_model.add(Input(shape=(112, 112, 3)))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(video_features, batch_size=16, epochs=10)

На данный момент решаю проблему с None значениями в тензорах при парсинге датасета. Пробовал использовать как и первичный датасет Slovo, так и resize версию, а также делать обработку ошибок на разных этапах предобработки видео, пока что значения None в выборках сохраняются
